In [1]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

# Data

In [2]:
nltk.download('treebank')


[nltk_data] Downloading package treebank to
[nltk_data]     /Users/wuxiaopan/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
nltk.download('universal_tagset')


[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/wuxiaopan/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
# load the Treebank tagged sentences
nlkt_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))


In [5]:
nlkt_data[:1]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')]]

# Training Data

In [6]:
# split data
train_set,test_set = train_test_split(nlkt_data, train_size=.8, test_size=.2, random_state=2)


In [7]:
# flatten data
train_words_tag_set = [ t for segm in train_set for t in segm ]

test_words_tag_set = [ t for segm in test_set for t in segm ]


In [8]:
len(train_words_tag_set)

80351

In [9]:
len(test_words_tag_set)

20325

In [10]:
train_words_tag_set[:3]

[('Sumitomo', 'NOUN'), ('Metal', 'NOUN'), ('Mining', 'NOUN')]

# Extract Tags

In [11]:
# transform to numpy array for easy computation
s_train_words_tag_set = np.array(train_words_tag_set)


In [12]:
# unique tags
pos_tags = np.unique(s_train_words_tag_set[:, 1])

len(pos_tags), pos_tags

(12,
 array(['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON',
        'PRT', 'VERB', 'X'], dtype='<U24'))

In [13]:
vocabulary = s_train_words_tag_set[:,0]

# vocabulary
len(vocabulary), vocabulary[: 3]


(80351, array(['Sumitomo', 'Metal', 'Mining'], dtype='<U24'))

# Functions

$$
p(z_n|z_{n-1}) = \frac{Count(z_n, z_{n-1})}{Count(z_{n-1})} 
\\
p(x|z_n) = \frac{Count(z_n, x)}{Count(z_{n})}
$$

In [14]:
def z2_given_z1(z2, z1, table):    
    """
    table: [(word, tag)]
    """
    tags = table[:, 1]
    len_z1 = (tags == z1).sum()
    len_z2_z1 = 0
    for i in range(len(tags) - 1):
        if tags[i] == z1 and tags[i+1] == z2:
            len_z2_z1 += 1
    
    return len_z2_z1 / len_z1


def word_given_state(word, z1, table):
    tags = table[:, 1]
    len_z1 = (tags == z1).sum()

    z1_table = table[tags == z1]
    len_word = (z1_table[:, 0] == word).sum()
    
    return len_word / len_z1


## transition matrix
def create_transition_matrix(word_tag_table, tag_list):
    matrix = np.zeros((len(tag_list), len(tag_list)))

    for i, z_1 in enumerate(tag_list):
        for j, z_2 in enumerate(tag_list):
            matrix[i, j] = z2_given_z1(z_2, z_1, word_tag_table)

    return matrix


In [15]:
def Vibiterbi(words, pos_tags, transition_matrix, word_tag_table):
    T = len(words) # the length of the observations
    N = len(pos_tags) # the length of the state

    prob_matrix = np.zeros((N, T))
    backpointer = np.zeros((N, T), dtype=int)

    # initilization state
    backpointer[:, 0] = 0

    for word_index in range(T): # for each word
        for state_index in range(N): # for each state 
            z_2 = pos_tags[state_index]
            emission_prob = word_given_state(words[word_index], z_2, word_tag_table)
            
            # initilization state
            if word_index == 0:
                transition_prob = transition_matrix.loc['.', z_2]
                prob_matrix[state_index, word_index] = transition_prob * emission_prob
            else:
                p = []
                z1_prob_list = prob_matrix[:, word_index - 1] # N state probability
                for z1_index, z1_prob in enumerate(z1_prob_list):
                    p.append(z1_prob * transition_matrix.loc[pos_tags[z1_index], z_2] * emission_prob)

                max_prob = max(p)
                max_prob_index = np.argmax(p)

                prob_matrix[state_index, word_index] = max_prob
                backpointer[state_index, word_index] = max_prob_index

    bestprob = np.max(prob_matrix[:,-1])
    bestpointer = np.argmax(prob_matrix[:,-1])

    bestpath = []
    for i in range(T): 
        bestpath.append(pos_tags[bestpointer])
        bestpointer = backpointer[bestpointer, -1 - i]

    return bestprob, bestpath[::-1]


# Transition matrix

In [16]:
transition_matrix = create_transition_matrix(s_train_words_tag_set, pos_tags)


In [17]:
df_transition_matrix = pd.DataFrame(transition_matrix, columns=pos_tags, index=pos_tags)


# Predict

In [107]:
test_sent = "Peter is a young man , A company market is raising"

_, test_tags = Vibiterbi(test_sent.split(), pos_tags, df_transition_matrix, s_train_words_tag_set)

test_tags


['NOUN',
 'VERB',
 'DET',
 'ADJ',
 'NOUN',
 '.',
 'DET',
 'NOUN',
 'NOUN',
 'VERB',
 'VERB']

In [19]:
s_test_words_tag_set = np.array(test_words_tag_set)


In [98]:
y_test = s_test_words_tag_set[:, 1][:29]

X_test = s_test_words_tag_set[:, 0][:29]


In [99]:
_, y_hat = Vibiterbi(X_test, pos_tags, df_transition_matrix, s_train_words_tag_set)


In [100]:
# accuracy: If the test words are not in the vocabulary, the accuracy is very low.
np.sum(y_hat == y_test) / len(y_test)


1.0